In [2]:
# import getpass
import os
import bs4
os.environ['USER_AGENT'] = 'myagent'
from langchain import hub
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# os.environ["LANGSMITH_TRACING"] = "false"

In [3]:
embeddings = OllamaEmbeddings(model="deepseek-r1:70b")
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_db/",
)

In [17]:
# Load and chunk contents of the blog
#loader = WebBaseLoader(
#    web_paths=("http://www.dengzhou.gov.cn/dzszj/zjxw/zwyw/webinfo/2024/11/1723956574402713.htm",),
#    bs_kwargs=dict(
#        parse_only=bs4.SoupStrainer(
#            class_=("post-content", "post-title", "post-header")
#        )
#    ),
#)
libdir = "../library/MIT"
papers = []
all_splits = []
for f in os.listdir(libdir):
    if f.endswith(".pdf"):
        papers.append(path.abspath(path.join(libdir, f)))
for f in papers:
    loader = PyMuPDFLoader(path.normpath(f))
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
    all_splits += text_splitter.split_documents(docs)

  Loading /Users/huo/Projects/ai4s/library/MIT/51279136-MIT-Spacecraft formation flight exploiting potential fields.pdf...
  Loading /Users/huo/Projects/ai4s/library/MIT/36829799-MIT-A combination of structurally connected and multiple spacecraft interferometers.pdf...
  Loading /Users/huo/Projects/ai4s/library/MIT/49870926-MIT-A structurally coupled disturbance analysis method using dynamic mass measurement techniques, with application to spacecraft-reaction wheel systems.pdf...
  Loading /Users/huo/Projects/ai4s/library/MIT/56544294-MIT-Integrated modeling of optical performance for the Terrestrial Planet Finder structurally connected interferometer.pdf...
  Loading /Users/huo/Projects/ai4s/library/MIT/54091966-MIT-Optical-level structural modelling of membrane mirrors for spaceborne telescopes.pdf...
  Loading /Users/huo/Projects/ai4s/library/MIT/1061505769-MIT-Fine-pointing system development and photometric precision assessment for the transiting exoplanet survey satellite.pdf...


In [18]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [4]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOllama(
    model="deepseek-r1:70b",
    temperature=0.0,
    num_predict=-1
)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [7]:
response = graph.invoke({"question": "compare structurally connected interferometer and separated interferometer"})
print(response["answer"])

<think>
Okay, so I need to compare structurally connected interferometers and separated interferometers based on the provided context. Let me go through the context step by step.

First, looking at section 5.6.4 about formation flying spacecraft, it mentions that free-flying systems are inherently uncoupled. This suggests that separated interferometers don't have a physical connection between the components, which makes sense because they're on different spacecraft. The synchronization strategy here involves maintaining a desired baseline length, which is crucial for interferometry as it affects resolution.

In contrast, structurally connected interferometers would have their components physically linked, perhaps on the same platform or structure. This likely provides mechanical stability and easier alignment since everything is fixed in place. However, this setup might limit flexibility, especially if you need to adjust the baseline length for different observations.

The context also